In [1]:
import numpy as np
import pandas as pd
from math import pow

In [6]:
    df = pd.read_csv('data/athlete_events.csv', sep=',')

    df = df.dropna(subset=['Medal'])
    df = df.dropna(subset=['Height'])
    df = df.dropna(subset=['Weight'])
    df = df.query("Team != 'China'")

#     copy = pd.read_csv('data/athlete_events.csv', sep=',')

    copy = copy.dropna(subset=['Medal'])
    copy = copy.dropna(subset=['Height'])
    copy = copy.dropna(subset=['Weight'])
    copy = copy.query("Team != 'China'")
    
#     # copy_count = copy['Year'].count()
#     # for i in range(copy_count):
#     #     copy.loc[copy.ID == i,'Year'] += 1000
    copy['Year'] = copy['Year'].apply(lambda x : x+1000)

#     # list(df['ID'])
#     # # prin  t(yl)
#     # copy = pd.DataFrame({
#     #     "ID":list(df['ID']),
#     #     "Name": list(df['Name']),
#     #     "Sex": list(df['Sex']),
#     #     "Age": list(df['Age']),
#     #     "Height": list(df['Height']),
#     #     "Weight": list(df['Weight']),
#     #     "Team": (df['Team']),
#     #     "NOC": list(df['NOC']),
#     #     "Games": list(df['Games']),
#     #     "Year": yl,
#     #     "Season": list(df['Season']),
#     #     "City": list(df['City']),
#     #     "Sport": list(df['Sport']),
#     #     "Event": list(df['Event']),
#     #     "Medal": list(df['Medal'])
#     # })
#     # print(copy[:50])


    df = df.append(copy,ignore_index=True
    # PARTICIPANTS DATAFRAME PREPARE
    raw_participants = df.loc[:, 'Name':'Team'] # взять часть столбцов из таблицы (срез по столбцам)
    participants = raw_participants.drop_duplicates(subset='Name', keep='last') # удалить строчки с одинаковыми именами
    participants = participants.reset_index(drop=True) # переиндексируем

    # GAMES DATAFRAME PREPARE
    raw_games = df.loc[:, 'Year':'City']
    games = raw_games.drop_duplicates()
    games = games.reset_index(drop=True)
    
    # рандомим дополнительные олимпийские игры 
   # games_count = games['Year'].count()
   # fake_cities_of_games = list(games['City'])
   # fake_years_of_games = [str(2018 + 2*i) for i in range(games_count)]
   # seasons = ['Winter','Summer']

   # val = pd.DataFrame([ [fake_years_of_games[i], seasons[i%2], fake_cities_of_games[i]] for i in range(games_count)],
   #                                                                                     columns=['Year', 'Season', 'City'])

   # games = games.append(val, ignore_index=True) # Без присвоения append не сработает

    # EVENTS DATAFRAME PREPARE
    events = df.loc[:, 'Sport':'Event']
    events = events.drop_duplicates(subset='Event', keep='last')
    events = events.reset_index(drop=True)

    list_of_events = events['Event'].values
    count_of_participants_in_event = [df[df['Event'] == event]['Name'].count() for event in list_of_events]
    events.insert(loc=len(events.columns), column='PCount', value=count_of_participants_in_event)
    events = events.reset_index(drop=True)
    events

    # PREPARE PARTICIPANT-EVENT RELATION
    pe_name = df['Name']
    pe_events = df['Event']
    pe = pd.DataFrame({
        'Event': df['Event'],
        'Name': df['Name']
    })
    pe = pe.drop_duplicates()
    pe = pe.reset_index(drop=True)

    list_of_participants_ids = []
    list_of_events_ids = []
    list_of_participants = list(participants['Name'].values)
    print(len(list_of_participants))
    for man in list_of_participants:
        list_of_events_of_participant = list(pe[pe['Name'] == man]['Event'].values)
        for ev in list_of_events_of_participant:
            print(man, ev,'\n')
            list_of_participants_ids.append(participants[participants['Name'] == man].index[0]+1)
            list_of_events_ids.append(events[events['Event'] == ev].index[0]+1)


    participant_event_relation_table = pd.DataFrame({
        'ParticipantID': list_of_participants_ids,
        'EventID': list_of_events_ids
    })

    participant_event_relation_table = participant_event_relation_table.sort_values(by='ParticipantID')
    participant_event_relation_table = participant_event_relation_table.drop_duplicates()
    participant_event_relation_table 

    # PREPARE EVENT-GAME RELATION
    eg_events = df['Event']
    eg_game = df['Year']
    eg_city = df['City']

    eg = pd.DataFrame({
        'Event': eg_events,
        'Game': eg_game,
        'City': eg_city
    })
    eg = eg.drop_duplicates()
    eg = eg.sort_values(by=['Game'])
    eg = eg.reset_index(drop=True)
    
    list_of_games_ids = []
    list_of_events_ids = []
    list_of_games = list(games['Year'].values)
    list_of_cities = list(games['City'].values)
    eg_count = games['Year'].count()

    for i in range(eg_count):
        list_of_events_in_game = list(eg[(eg['Game'] == list_of_games[i]) & (eg['City'] == list_of_cities[i])]['Event'].values)
        for ev in list_of_events_in_game:
            list_of_games_ids.append(games[(games['Year'] == list_of_games[i]) & (games['City'] == list_of_cities[i])].index[0]+1) # TO DO добавить выборку по городу
            list_of_events_ids.append(events[events['Event'] == ev].index[0]+1)

    event_game_relation_table = pd.DataFrame({
        'GameID': list_of_games_ids,
        'EventID': list_of_events_ids
    })

    event_game_relation_table = event_game_relation_table.sort_values(by='GameID')
    event_game_relation_table = event_game_relation_table.drop_duplicates()
    event_game_relation_table

    # PREPARE GAME-PARICIPANT RELATION
    gp_game = df['Year']
    gp_name = df['Name']
    gp_city = df['City']

    gp = pd.DataFrame({
        'Game': gp_game,
        'Name': gp_name,
        'City': gp_city
    })
    gp = gp.drop_duplicates()

    list_of_participants_ids = []
    list_of_games_ids = []
    list_of_participants = list(participants['Name'].values)

    for man in list_of_participants:
        list_of_games_of_participant = list(gp[gp['Name'] == man]['Game'].values)
        list_of_cities_of_participant = list(gp[gp['Name'] == man]['City'].values)
        games_count = len(list_of_games_of_participant)

        for i in range(games_count):
            list_of_participants_ids.append(participants[participants['Name'] == man].index[0]+1)
            list_of_games_ids.append(games[(games['Year'] == list_of_games_of_participant[i]) & (games['City'] == list_of_cities_of_participant[i])].index[0]+1)


    game_participant_relation_table = pd.DataFrame({
        'ParticipantID': list_of_participants_ids,
        'GameID': list_of_games_ids
    })
    game_participant_relation_table = game_participant_relation_table.sort_values(by='GameID')
    game_participant_relation_table = game_participant_relation_table.drop_duplicates()
    game_participant_relation_table  

    return participants, games, events, participant_event_relation_table, event_game_relation_table, game_participant_relation_table


In [7]:
create

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
1,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
2,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold
3,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold
4,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Pommelled Horse,Gold
5,17,Paavo Johannes Aaltonen,M,32.0,175.0,64.0,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Gymnastics,Gymnastics Men's Team All-Around,Bronze
6,20,Kjetil Andr Aamodt,M,20.0,176.0,85.0,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Super G,Gold
7,20,Kjetil Andr Aamodt,M,20.0,176.0,85.0,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Giant Slalom,Bronze
8,20,Kjetil Andr Aamodt,M,22.0,176.0,85.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Men's Downhill,Silver
9,20,Kjetil Andr Aamodt,M,22.0,176.0,85.0,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Men's Super G,Bronze
